In [5]:
# import dill
import scanpy as sc
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

# import pyranges
# Set stderr to null to avoid strange messages from ray
import sys
_stderr = sys.stderr
null = open(os.devnull,'wb')
from scipy.sparse import issparse


import itertools
import anndata
import itertools
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import warnings
warnings.simplefilter(action='ignore')
import numpy as np
from sklearn.preprocessing import normalize

from itertools import product
import umap
from collections import Counter
from sklearn.decomposition import PCA
from gridot.utils import *
from gridot.train import *

In [6]:
atac_anndata = sc.read_h5ad('./data/pseudo_atac_20_0.1.h5ad')
atac_anndata

AnnData object with n_obs × n_vars = 1879 × 113233
    obs: 'celltype', 'n_genes', 'orig.ident'
    var: 'peaks', 'chr_no', 'start', 'end', 'Chromosome', 'Start', 'End', 'n_cells', 'features'

In [7]:
atac_anndata.obs['celltype'].unique()

['DCT', 'FIB', 'ENDO', 'CNT_PC', 'LEUK']
Categories (5, object): ['CNT_PC', 'DCT', 'ENDO', 'FIB', 'LEUK']

In [8]:
rna_anndata = sc.read_h5ad('./data/rna_data_filtered_pca_1control_5cells_real.h5ad')

# Check if raw data exists
if rna_anndata.raw is not None:
    # Access the raw data
    raw_data = rna_anndata.raw.X
    raw_obs = rna_anndata.obs.copy()  # Cell metadata (unchanged)
    raw_var = rna_anndata.raw.var.copy()  # Gene metadata (original raw genes)

    # Create a new AnnData object using the raw data
    rna_anndata = sc.AnnData(X=raw_data, obs=raw_obs, var=raw_var)

    # Print the new raw AnnData object to verify
    print(rna_anndata)
else:
    print("The AnnData object does not contain raw data.")

The AnnData object does not contain raw data.


In [9]:
folder_path = f"./results/CRE_gene"
schema_reference = 'rna'
marker_gene = 'PAX2'
distance_threshold = 1e5
device = 'cuda:0'
num_components = 50
gene_file = "./data/gencode.v38.annotation.gff3"


In [10]:
import os

schema_reference = 'rna'
marker_gene = 'PAX2'
distance_threshold = 1e5
device = 'cuda:0'
num_components = 50
gene_file = "./data/gencode.v38.annotation.gff3"
base_folder_path = './testing_results_pseudo'
seeds = [1]

# Preprocess once
atac_adata, rna_adata = preprocessing_adata(atac_anndata, rna_anndata, gene_file=gene_file)
print("Processing multimodal")
preprocess_multimodal(rna_adata, atac_adata)

# Run for each seed
for seed in seeds:
    seed_folder_path = os.path.join(base_folder_path, f"seed_{seed}")
    os.makedirs(seed_folder_path, exist_ok=True)
    print(f"Running CRE-gene relationship analysis with seed {seed}")
    run_gridot_cell_population(
        rna_adata,
        atac_adata,
        schema_reference=schema_reference,
        marker_gene=marker_gene,
        distance_threshold=distance_threshold,
        device=device,
        folder_path=seed_folder_path,
        seed=seed
    )


Pre-processing scATAC-seq...
AnnData object with n_obs × n_vars = 1879 × 113233
    obs: 'celltype', 'n_genes', 'orig.ident'
    var: 'peaks', 'chr_no', 'start', 'end', 'Chromosome', 'Start', 'End', 'n_cells', 'features'
Pre-processing scRNA-seq...
AnnData object with n_obs × n_vars = 1879 × 20496
    obs: 'name', 'barcode', 'patient', 'gender', 'disease', 'celltype', 'nCount_RNA', 'nFeature_RNA', 'UMAP_1', 'UMAP_2', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'features'
Adding start site to meta file...
Processing multimodal
Running CRE-gene relationship analysis
Identifying candidate pairs within 100000.0 bp
188306
Running quadratic program... done.
Cell with highest PAX2 expression: ('TTCCGTGCAGCTGCCA-1', 1785)
Expression level:   (0, 0)	5.677961297120604
Epoch 0 (8.48 seconds): train loss 671.21
Epoch Inference (19.18 seconds): test loss 655.